## Decision Tree Example

In [4]:
import numpy as np
import pandas as pd 
from sqlalchemy import create_engine ,cast
# from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

from graphviz import Source
from sklearn import tree
from sklearn.tree import export_graphviz
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
#Source( tree.export_graphviz(dtreg, out_file=None, feature_names=X.columns))
# from sklearn import tree
# pip install graphviz
import graphviz

### 1. 분석데이터 로딩

In [5]:
#CSV 파일을 읽어 DataFrame 변수에 저장하기
featuresData = pd.read_csv("../dataset./feature_regression_example.csv")
featuresData.head()

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2,968,N,4,Y,0.209442
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3,1209,N,4,Y,0.208155
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4,1810,Y,2,Y,0.208155
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5,1773,N,4,Y,0.208155


In [34]:
featuresData.head(1)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PROMOTIONCODE,HOLIDAYCODE
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1.0,1225.0,Y,1,Y,0.209442,1,1


### 2.데이터 형 변환

In [36]:
featuresData.dtypes # 데이터의 타입을 다 볼 수 있다. 

REGIONID          object
PRODUCTGROUP      object
PRODUCT           object
ITEM              object
YEARWEEK           int64
YEAR               int64
WEEK             float64
QTY              float64
HOLIDAY           object
HCLUS              int64
PROMOTION         object
PRO_PERCENT      float64
PROMOTIONCODE      int64
HOLIDAYCODE        int64
dtype: object

In [38]:
featuresData[['WEEK','QTY','PRO_PERCENT']]

,WEEK,QTY,PRO_PERCENT
0,1.0,1225.0,0.209442
1,2.0,968.0,0.209442
2,3.0,1209.0,0.208155
3,4.0,1810.0,0.208155
4,5.0,1773.0,0.208155
5,6.0,867.0,0.208155
6,7.0,1187.0,0.208155
7,8.0,970.0,0.208155
8,9.0,542.0,0.208155
9,10.0,350.0,0.000000


In [6]:
#형변환
featuresData[['WEEK','QTY','PRO_PERCENT']]=featuresData[['WEEK','QTY','PRO_PERCENT']].astype('float64')
featuresData.dtypes

REGIONID         object
PRODUCTGROUP     object
PRODUCT          object
ITEM             object
YEARWEEK          int64
YEAR              int64
WEEK            float64
QTY             float64
HOLIDAY          object
HCLUS             int64
PROMOTION        object
PRO_PERCENT     float64
dtype: object

### 3. 문자데이터 코드변환(Vector연산)

In [7]:
#case when 
############################
def codeConversion(df):
    if df == "Y":
        return 1
    else:
        return 0
    
############################       
    
featuresData['PROMOTIONCODE'] = featuresData['PROMOTION'].apply(codeConversion)
featuresData.head()

featuresData['HOLIDAYCODE'] = featuresData['HOLIDAY'].apply(codeConversion)
featuresData.head()

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PROMOTIONCODE,HOLIDAYCODE
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1.0,1225.0,Y,1,Y,0.209442,1,1
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2.0,968.0,N,4,Y,0.209442,1,0
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3.0,1209.0,N,4,Y,0.208155,1,0
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4.0,1810.0,Y,2,Y,0.208155,1,1
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5.0,1773.0,N,4,Y,0.208155,1,0


In [50]:
featuresData['HOLIDAYCODE']

0      1
1      0
2      0
3      1
4      0
5      0
6      0
7      1
8      0
9      0
10     0
11     0
12     0
13     0
14     0
15     0
16     0
17     0
18     0
19     1
20     1
21     1
22     0
23     0
24     1
25     1
26     1
27     1
28     1
29     0
      ..
77     1
78     1
79     1
80     1
81     1
82     0
83     0
84     0
85     0
86     1
87     1
88     1
89     0
90     0
91     0
92     0
93     1
94     0
95     0
96     1
97     1
98     1
99     1
100    1
101    0
102    0
103    0
104    1
105    0
106    0
Name: HOLIDAYCODE, Length: 107, dtype: int64

In [49]:
featuresData.head()

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PROMOTIONCODE,HOLIDAYCODE
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1.0,1225.0,Y,1,Y,0.209442,1,1
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2.0,968.0,N,4,Y,0.209442,1,0
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3.0,1209.0,N,4,Y,0.208155,1,0
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4.0,1810.0,Y,2,Y,0.208155,1,1
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5.0,1773.0,N,4,Y,0.208155,1,0


### 4. 데이터 셋 분리

In [53]:
predictStd = 201630 # 변수를 하나 설정. 
                # 기준이 되는 주차 정보값을 입력 (어디서 부터 트레이닝데이터로 쓸 것인지 )

In [52]:
featuresData

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PROMOTIONCODE,HOLIDAYCODE
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1.0,1225.0,Y,1,Y,0.209442,1,1
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2.0,968.0,N,4,Y,0.209442,1,0
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3.0,1209.0,N,4,Y,0.208155,1,0
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4.0,1810.0,Y,2,Y,0.208155,1,1
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5.0,1773.0,N,4,Y,0.208155,1,0
5,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201506,2015,6.0,867.0,N,4,Y,0.208155,1,0
6,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201507,2015,7.0,1187.0,N,4,Y,0.208155,1,0
7,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201508,2015,8.0,970.0,Y,1,Y,0.208155,1,1
8,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201509,2015,9.0,542.0,N,4,Y,0.208155,1,0
9,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201510,2015,10.0,350.0,N,4,N,0.000000,0,0


In [57]:
#where between and (조건 절)
trainingData = featuresData.query('YEARWEEK <= @predictStd')
trainingData.head() # 여기까지 트레이닝 데이터 

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PROMOTIONCODE,HOLIDAYCODE
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1.0,1225.0,Y,1,Y,0.209442,1,1
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2.0,968.0,N,4,Y,0.209442,1,0
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3.0,1209.0,N,4,Y,0.208155,1,0
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4.0,1810.0,Y,2,Y,0.208155,1,1
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5.0,1773.0,N,4,Y,0.208155,1,0


In [55]:
#where between and (조건 절)
testData = featuresData.query('YEARWEEK > @predictStd')
testData.head() # 여기까지 테스트 데이터 

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PROMOTIONCODE,HOLIDAYCODE
83,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201631,2016,31.0,1522.0,N,4,Y,0.280258,1,0
84,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201632,2016,32.0,2100.0,N,4,Y,0.280258,1,0
85,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201633,2016,33.0,43.0,N,4,N,0.000000,0,0
86,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201634,2016,34.0,1700.0,Y,1,Y,0.308584,1,1
87,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201635,2016,35.0,1514.0,Y,1,Y,0.308584,1,1


In [11]:
trainingData_feature = trainingData[['WEEK','PRO_PERCENT','HOLIDAYCODE']] #트레이닝 데이터의 피처를 분리

In [12]:
trainingData_label = trainingData[['QTY']] # 트레이닝 데이터의 라벨을 분리 

In [13]:
testData_feature = testData[['WEEK','PRO_PERCENT','HOLIDAYCODE']] # 마찬가지로 테스트 데이터도 피처를 분리 

In [14]:
testData_label = testData[['QTY']] # 테스트 데이터 셋의 라벨을 분리 

In [15]:
# from sklearn import tree
#clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)

### 5. 모델선언 및 예측

In [16]:
#model_method= tree.DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=0)

In [17]:
# from sklearn import tree
#model_method = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)

In [59]:
# from sklearn import tree
#model_method = tree.DecisionTreeClassifier(random_state=0) 
model_method = tree.DecisionTreeClassifier() # 어떤 모델을 쓸지 선언 

In [ ]:
## Extrace Coefficient # 내가 이 머신 러닝을 돌릴때 피처만 넣으면 알아서 예측하는 식으로 ...


In [64]:
model = model_method.fit(trainingData_feature, trainingData_label) # 많이 보던 fit 메소드 ...


In [65]:
predict = model.predict(testData_feature) # 예측단에서는 라벨은 들어가지 않고 피쳐 데이터만 넣어서 예측 수행


In [66]:
predict # 예측치 

array([2240., 2240.,  147., 1318., 1318., 1318., 2240., 2240., 2240.,
       2240., 2671., 2240.,  230., 1454., 1454., 2033., 2033., 3691.,
       1758., 1758., 1758., 1758.,  350.,  301.])

In [63]:
testData_label.head() # 정답지. 

,QTY
83,1522.0
84,2100.0
85,43.0
86,1700.0
87,1514.0


### 6. 데이터 정리

In [68]:
type(predict) #숫자의 조합으로 나오는 경우 type으로 어떤 타입인지 확인 가능 (여기서는 숫자의 array로 확인 가능 )

numpy.ndarray

In [23]:
predictData = pd.DataFrame(predict)

In [71]:
predictData

,PREDICT
0,2240.0
1,2240.0
2,147.0
3,1318.0
4,1318.0
5,1318.0
6,1318.0
7,1318.0
8,2240.0
9,2240.0


In [72]:
predictData.columns = ["PREDICT"] # 컬럼이름 명시 

In [73]:
testData #기존의 인덱스 번호를 가지고 있는 것을 reset 인덱스를 하자

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PROMOTIONCODE,HOLIDAYCODE
83,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201631,2016,31.0,1522.0,N,4,Y,0.280258,1,0
84,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201632,2016,32.0,2100.0,N,4,Y,0.280258,1,0
85,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201633,2016,33.0,43.0,N,4,N,0.000000,0,0
86,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201634,2016,34.0,1700.0,Y,1,Y,0.308584,1,1
87,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201635,2016,35.0,1514.0,Y,1,Y,0.308584,1,1
88,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201636,2016,36.0,1501.0,Y,1,Y,0.308584,1,1
89,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201637,2016,37.0,1491.0,N,4,Y,0.308584,1,0
90,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201638,2016,38.0,806.0,N,4,Y,0.308584,1,0
91,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201639,2016,39.0,2111.0,N,4,Y,0.280258,1,0
92,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201640,2016,40.0,2400.0,N,4,Y,0.280258,1,0


In [74]:
testData.reset_index(drop=True, inplace=True)

In [75]:
testData # 위의 코드를 수행함으로 인해 인덱스 값이 reset 됨 

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PROMOTIONCODE,HOLIDAYCODE
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201631,2016,31.0,1522.0,N,4,Y,0.280258,1,0
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201632,2016,32.0,2100.0,N,4,Y,0.280258,1,0
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201633,2016,33.0,43.0,N,4,N,0.000000,0,0
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201634,2016,34.0,1700.0,Y,1,Y,0.308584,1,1
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201635,2016,35.0,1514.0,Y,1,Y,0.308584,1,1
5,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201636,2016,36.0,1501.0,Y,1,Y,0.308584,1,1
6,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201637,2016,37.0,1491.0,N,4,Y,0.308584,1,0
7,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201638,2016,38.0,806.0,N,4,Y,0.308584,1,0
8,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201639,2016,39.0,2111.0,N,4,Y,0.280258,1,0
9,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201640,2016,40.0,2400.0,N,4,Y,0.280258,1,0


In [26]:
predictData.reset_index(drop=True, inplace=True)

In [27]:
#testData_feature.reset_index(drop=True, inplace=True)

In [76]:
#cncatenate labels to df as a new column / column binding
finalDf = pd.concat([testData, predictData], axis = 1) # 테스트 데이터와 예측 데이터를 옆에 붙이겠다. 

In [77]:
finalDf.head()

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PROMOTIONCODE,HOLIDAYCODE,PREDICT
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201631,2016,31.0,1522.0,N,4,Y,0.280258,1,0,2240.0
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201632,2016,32.0,2100.0,N,4,Y,0.280258,1,0,2240.0
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201633,2016,33.0,43.0,N,4,N,0.000000,0,0,147.0
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201634,2016,34.0,1700.0,Y,1,Y,0.308584,1,1,1318.0
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201635,2016,35.0,1514.0,Y,1,Y,0.308584,1,1,1318.0


In [80]:
predictData

,PREDICT
0,2240.0
1,2240.0
2,147.0
3,1318.0
4,1318.0
5,1318.0
6,1318.0
7,1318.0
8,2240.0
9,2240.0


In [82]:
pwd

'C:\\Users\\cj\\Python_Working\\Python_CJ_ST\\Session01 - Why Python for Data Analysis'

In [83]:
finalDf.to_csv("./dt_result_20181022.csv")

### 7. 정확도 측정

In [30]:
mean_absolute_error(finalDf['QTY'], finalDf['PREDICT'])  # 차이가 


376.2083333333333

In [31]:
mean_squared_error(finalDf['QTY'], finalDf['PREDICT'])  

263635.4583333333

In [32]:
r2_score(finalDf['QTY'], finalDf['PREDICT'])  

0.5703348004273314

In [33]:
finalDf.to_csv("./decisiontree_result.csv")